In [5]:
import pandas as pd
import sklearn

In [3]:
pd.__version__

'1.4.2'

In [6]:
!pip install pyarrow

     |████████████████████████████████| 40.8 MB 18.3 MB/s eta 0:00:01


In [3]:
df= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [36]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [12]:
df.columns()

AttributeError: 'DataFrame' object has no attribute 'column'

In [31]:
cols = len(df.axes[1])

In [32]:
cols

19

### First Question answer is 19

In [15]:
# Counting NaN values in all columns
nan_count = df.isna().sum()

print(nan_count)


VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          140162
trip_distance                 0
RatecodeID               140162
store_and_fwd_flag       140162
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     140162
Airport_fee              140162
dtype: int64


In [19]:
row = len(df.axes[0])

In [20]:
row

2824462

In [18]:

# drop columns with all NaN's
df = df.dropna(axis=0)

In [21]:
x=2964624 -2824462

In [22]:
x

140162

In [4]:
# Convert pickup and dropoff datetime columns to datetime objects
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Compute the trip duration in minutes
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter trips for January
january_trips = df[df['tpep_pickup_datetime'].dt.month == 1]

# Calculate the standard deviation of trip durations in January
std_dev_duration_january = january_trips['trip_duration'].std()

print("Standard deviation of trip durations in January:", std_dev_duration_january)

Standard deviation of trip durations in January: 34.85113736077084


In [5]:
# Filter records where duration is between 1 and 60 minutes (inclusive)
filtered_data = df[(df['trip_duration'] >= 1) & (df['trip_duration'] <= 60)]

# Calculate fraction of records left after dropping outliers
fraction_left = len(filtered_data) / len(df) * 100

print("Fraction of records left after dropping outliers:", fraction_left, "%")


Fraction of records left after dropping outliers: 97.78326020432945 %


In [6]:
from sklearn.feature_extraction import DictVectorizer

# Convert dataframe to a list of dictionaries with IDs casted to strings
data_dicts = filtered_data[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize a dictionary vectorizer
vectorizer = DictVectorizer(sparse=False)

# Fit the vectorizer and transform the data to obtain the feature matrix
feature_matrix = vectorizer.fit_transform(data_dicts)

# Get the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]

print("Dimensionality of the feature matrix (number of columns):", num_columns)


MemoryError: Unable to allocate 11.2 GiB for an array with shape (2898906, 518) and data type float64

In [7]:
from sklearn.feature_extraction import DictVectorizer

# Convert dataframe to a list of dictionaries with IDs casted to strings
data_dicts = filtered_data[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize a dictionary vectorizer with sparse output
vectorizer = DictVectorizer(sparse=True)

# Fit the vectorizer and transform the data to obtain the feature matrix
feature_matrix = vectorizer.fit_transform(data_dicts)

# Get the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]

print("Dimensionality of the feature matrix (number of columns):", num_columns)


Dimensionality of the feature matrix (number of columns): 518


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the feature matrix (from previous step) and the target variable
X = feature_matrix
y = df['trip_duration']

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Make predictions on the training data
predictions = model.predict(X)

# Calculate RMSE on the training data
rmse = np.sqrt(mean_squared_error(y, predictions))

print("RMSE on training data:", rmse)


ValueError: Found input variables with inconsistent numbers of samples: [2898906, 2964624]

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Ensure the duration is already computed
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter records where duration is between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Convert filtered dataframe to a list of dictionaries with IDs casted to strings
data_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize a dictionary vectorizer
vectorizer = DictVectorizer(sparse=False)

# Fit the vectorizer and transform the data to obtain the feature matrix
X = vectorizer.fit_transform(data_dicts)

# Prepare the target variable
y = df_filtered['duration']

# Ensure X and y have the same number of samples
assert len(X) == len(y)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Make predictions on the training data
predictions = model.predict(X)

# Calculate RMSE on the training data
rmse = np.sqrt(mean_squared_error(y, predictions))

print("RMSE on training data:", rmse)


MemoryError: Unable to allocate 11.2 GiB for an array with shape (2898906, 518) and data type float64

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.sparse import csr_matrix

# Ensure the duration is already computed
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter records where duration is between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Convert filtered dataframe to a list of dictionaries with IDs casted to strings
data_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize a dictionary vectorizer with sparse=True
vectorizer = DictVectorizer(sparse=True)

# Fit the vectorizer and transform the data to obtain the feature matrix
X = vectorizer.fit_transform(data_dicts)

# Prepare the target variable
y = df_filtered['duration']

# Ensure X and y have the same number of samples
assert X.shape[0] == y.shape[0]

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Make predictions on the training data
predictions = model.predict(X)

# Calculate RMSE on the training data
rmse = np.sqrt(mean_squared_error(y, predictions))

print("RMSE on training data:", rmse)


RMSE on training data: 7.946173359562653


In [9]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.sparse import csr_matrix
vectorizer = DictVectorizer(sparse=True)
model = LinearRegression()
model.fit(X, y)


NameError: name 'X' is not defined

In [7]:
# Load the validation dataset
validation_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

# Ensure the duration is already computed
validation_df['tpep_pickup_datetime'] = pd.to_datetime(validation_df['tpep_pickup_datetime'])
validation_df['tpep_dropoff_datetime'] = pd.to_datetime(validation_df['tpep_dropoff_datetime'])
validation_df['duration'] = (validation_df['tpep_dropoff_datetime'] - validation_df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter records where duration is between 1 and 60 minutes (inclusive)
validation_df_filtered = validation_df[(validation_df['duration'] >= 1) & (validation_df['duration'] <= 60)]

# Convert filtered dataframe to a list of dictionaries with IDs casted to strings
validation_data_dicts = validation_df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Transform the validation data using the same vectorizer used for training
X_validation = vectorizer.transform(validation_data_dicts)
y_validation = validation_df_filtered['duration']

# Ensure X_validation and y_validation have the same number of samples
assert X_validation.shape[0] == y_validation.shape[0]

# Make predictions on the validation data
predictions_validation = model.predict(X_validation)

# Calculate RMSE on the validation data
rmse_validation = np.sqrt(mean_squared_error(y_validation, predictions_validation))

print("RMSE on validation data:", rmse_validation)


AttributeError: 'DictVectorizer' object has no attribute 'feature_names_'

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the validation dataset
validation_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the validation dataset

# Ensure the duration is already computed
validation_df['tpep_pickup_datetime'] = pd.to_datetime(validation_df['tpep_pickup_datetime'])
validation_df['tpep_dropoff_datetime'] = pd.to_datetime(validation_df['tpep_dropoff_datetime'])
validation_df['duration'] = (validation_df['tpep_dropoff_datetime'] - validation_df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter records where duration is between 1 and 60 minutes (inclusive)
validation_df_filtered = validation_df[(validation_df['duration'] >= 1) & (validation_df['duration'] <= 60)]

# Convert filtered dataframe to a list of dictionaries with IDs casted to strings
validation_data_dicts = validation_df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize a new dictionary vectorizer (using the same vocabulary as the one used for training)
vectorizer_validation = DictVectorizer(sparse=True)
X_validation = vectorizer_validation.fit_transform(validation_data_dicts)

# Prepare the target variable
y_validation = validation_df_filtered['duration']

# Ensure X_validation and y_validation have the same number of samples
assert X_validation.shape[0] == y_validation.shape[0]

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_validation, y_validation)

# Make predictions on the validation data
predictions_validation = model.predict(X_validation)

# Calculate RMSE on the validation data
rmse_validation = np.sqrt(mean_squared_error(y_validation, predictions_validation))

print("RMSE on validation data:", rmse_validation)



RMSE on validation data: 7.649261027919939
